In [1]:
%%bash

# Install required packages
pip install -U -q pandas scikit-learn imblearn

# Download & extract dataset
wget -q -O data.zip https://he-s3.s3.amazonaws.com/media/hackathon/predict-the-happiness/predict-the-happiness/f2c2f440-8-dataset_he.zip
unzip -oqq data.zip train.csv

  Failed building wheel for fasttext


In [0]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

### Загрузим данные, выровняем соотношение классов для упрощения задачи

In [3]:
df = pd.read_csv('train.csv', usecols=['Description', 'Is_Response'])\
  .rename(columns={'Description': 'x', 'Is_Response': 'y'})
df['y'] = df['y'].map(lambda x: x.replace(' ', '_'))

X, y = RandomUnderSampler(random_state=42).fit_resample(df[['x']], df['y'].values)
df = pd.DataFrame({'x': X[:, 0], 'y': y})

print(df.y.value_counts())
df.head()

happy        12411
not_happy    12411
Name: y, dtype: int64


,x,y
0,Henry is a big part of why I look forward to c...,happy
1,We stayed - nights at the blue sea lodge at th...,happy
2,I've stayed at several Embassy Suites in the p...,happy
3,I was recently in Chicago for business and sta...,happy
4,My wife and I recently traveled to NYC for fou...,happy


### Разобьем данные на обучающую и тестовую выборки

In [0]:
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['y'])

train_df = train_df.copy()
test_df = test_df.copy()

### Выполним векторизацию

[Раздел документации scikit-learn посвященный векторизации текстовых данных](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)


#### CountVectorizer
```Init signature: CountVectorizer(input='content', encoding='utf-8', decode_error='strict', strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words=None, token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1), analyzer='word', max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class 'numpy.int64'>)```

#### TfidfVectorizer
```Init signature: TfidfVectorizer(input='content', encoding='utf-8', decode_error='strict', strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, analyzer='word', stop_words=None, token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class 'numpy.float64'>, norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)```

In [8]:
vectorizer = 
train_X = vectorizer.fit_transform(train_df['x'].values)
test_X = vectorizer.transform(test_df['x'].values)


print(train_X.shape)

(19857, 1000)


### Обучим и провалидируем классификатор

In [9]:
classifier = 
classifier.fit(train_X, train_df['y'].values)

y_pred = classifier.predict(test_X)

print(classification_report(test_df['y'].values, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

       happy       0.74      0.83      0.78      2483
   not_happy       0.80      0.71      0.76      2482

   micro avg       0.77      0.77      0.77      4965
   macro avg       0.77      0.77      0.77      4965
weighted avg       0.77      0.77      0.77      4965



### Задание: собрать pipeline (vectorizer->classifier). Выполнить грид-поиск гиперпараметров

[Пример построения pipeline при решении задачи классификации текстов](https://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html#sphx-glr-auto-examples-model-selection-grid-search-text-feature-extraction-py)